In [1]:
# Standard Packages
from torch.utils.data import random_split, DataLoader
import torch
import wandb
from datetime import datetime
from matplotlib import pyplot as plt
import torch.nn as nn
# Custom Packages
from ssapp.data.AntennaDatasetLoaders import load_serialized_dataset
from ssapp.Utils import train_test_data_split

# General Settings
torch.manual_seed(42) # Manual seed for sanity
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")



In [16]:
## Define Model

NN = 1000

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.regressor = nn.Sequential(nn.Linear(3, NN),
                                       nn.ReLU(),
                                       nn.Linear(NN, NN),
                                       nn.ReLU(),
                                       nn.Linear(NN, 361*3*4))
        
    def forward(self, x):
        output = self.regressor(x)
        return output

In [12]:

dataset = load_serialized_dataset('PatchAntennaDataset2')


In [13]:
TEST_TRAIN_RATIO = 0.7
BATCH_SIZE = 1

train_set, test_set = train_test_data_split(dataset)

train_dataloader = DataLoader(train_set, batch_size = BATCH_SIZE, shuffle=True)
test_dataloader = DataLoader(test_set, batch_size = BATCH_SIZE, shuffle=True)



In [14]:
params,fields = next(iter(train_dataloader))
print(params.shape)

torch.Size([1, 3])


In [17]:

#Define model
model = Net()
#Test Forward pass
print(model(torch.randn(2,3).to(device)))

tensor([[-0.2309,  0.0827,  0.0967,  ..., -0.0139, -0.0969, -0.0800],
        [-0.1234,  0.0403, -0.0731,  ..., -0.0141, -0.0994,  0.0217]],
       grad_fn=<AddmmBackward0>)


In [27]:
LEARNING_RATE = 3e-4

criterion = torch.nn.MSELoss()

In [28]:
EPOCHS = int(200)

# Optional

optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)

startTime = datetime.now()
train_loss_array = []
test_loss_array = []
loss_array = []

for epoch in range(EPOCHS):

    epoch_training_loss = 0
    epoch_training_targets = 0
    train_batches = 0
    for train_params, train_fields in train_dataloader:
        
        ## Transfer Batch to Device
        train_params = train_params.float().to(device)
        train_fields = train_fields.float().to(device).reshape(len(train_fields),-1)
        
        prediction = model(train_params)
        

        loss = criterion(input = prediction, target = train_fields)
        epoch_training_loss += loss
        train_batches +=1
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    
    
    epoch_test_loss = 0
    epoch_test_targets = 0
    test_batches=0
    with torch.no_grad():
        
        for test_params, test_fields in test_dataloader:
            
            # Put thing to device
            test_params = test_params.float().to(device)
            test_fields = test_fields.float().to(device).reshape(len(test_fields),-1)
            
            
            
            prediction = model(test_params)
            loss = criterion(input = prediction, target = test_fields)
            epoch_test_loss += loss
            
            test_batches += 1
    

    print('Training Loss',(epoch_training_loss/train_batches).item())
    print('Test Loss',(epoch_test_loss/test_batches).item())
    

print('Training time:', datetime.now()-startTime)

KeyboardInterrupt: 

In [ ]:
inp, tar = next(iter(train_dataloader)) 
print(inp)
plt.figure()
plt.title('Training set example')
plt.plot(model(inp).flatten().detach(),label = 'Model Prediction')
plt.plot(tar.flatten(),label ='Truth')
plt.legend()

inp, tar = next(iter(test_dataloader)) 
plt.figure()
plt.title('Test set example')
plt.plot(model(inp).flatten().detach(),label = 'Model Prediction')
plt.plot(tar.flatten(),label = 'Truth')
plt.legend()

In [ ]:
## Test overfitting
EPOCHS = int(2e3)

overFitModel = DirectFeedForwardNet(in_features = 3,out_features = 4004,NN = 100)
optimizer = torch.optim.SGD(overFitModel.parameters(), lr=LEARNING_RATE, momentum=MOMENTUM)
inp, tar = next(iter(train_dataloader)) 
for epoch in range(EPOCHS):
    prediction = overFitModel(inp)
        
    loss = criterion(input = prediction, target = tar)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if epoch%(EPOCHS/10) == 0:
        print('Loss', loss.item())
    
    
plt.plot(overFitModel(inp).flatten().detach())
plt.plot(tar.flatten())

In [ ]:
inputs, classes = next(iter(train_dataloader)) 
print(inputs,classes)